In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
from lxml import etree
%aimport json
from pathlib import Path
%aimport re
from typing import List, Dict
%aimport os
from xml.sax.saxutils import escape
from pathlib import Path

In [5]:
from data_processing.text_processing import get_text_from_file, set_working_directory, get_working_directory
from data_processing.text_processing import normalize_quotes

In [6]:
from data_processing.text_processing import write_text_to_file

In [7]:
from data_processing.gpt_processing import token_count, set_api_client, get_api_client, get_active_batches
from data_processing.gpt_processing import generate_messages, run_immediate_chat_process, create_jsonl_file_for_batch, start_batch
from data_processing.gpt_processing import get_completed_batches, get_batch_response


In [8]:
from data_processing.xml_processing import split_xml_pages, save_pages_to_xml

In [9]:
project_dir = Path("/Users/phapman/Desktop/tnh-scholar/")
data_dir = project_dir / "data_processing"
journal_dir = data_dir / "processed_journal_data"

In [8]:
user_message_string_clean = """{text}"""

In [9]:
def user_wrap_function_clean(text_block):
    return user_message_string_clean.format(text=text_block)

In [10]:
system_message_clean = """
You are a meticulous and consistent world expert at cleaning OCR-generated Vietnamese text. 
You are cleaning text from a 1950's Buddhist Journal. 
The text will be given in XML, and the output text should be in matching XML. 
Your goal is to minimally modify the text to generate a cleaned version.
Do not remove any text from the main body of the text. 
Formatting markers (such as footers) at the end of the text can be adjusted or removed as needed for clarity. 
You can use patterns in the text blocks (given by page) to infer patterns in the text.
You can use the semantic meaning of the text to infer corrections—but make no semantic changes. 
You can also add diacritical marks if they are missing or clearly inaccurate. 
Do not change any proper names, except to add missing diacritical marks if the context is clear.  
This particular text has a title marker: "Phat Giao Viet Nam," and also a publishing mark near the end of each page of text. 
The publishing mark is something like "TU VIEN HUE QUANG"  and is very difficult for the OCR to process. 
Text corresponding to these marks (or part thereof) and page numbers can be omitted.
Output the corrected text only with no comments (including ``` xml).
"""

In [11]:
basename = "phat-giao-viet-nam-1956-01"
ocr_file_to_process = journal_dir / basename / f"full_OCR_{basename}.xml"
ocr_file_to_process

PosixPath('/Users/phapman/Desktop/tnh-scholar/data_processing/processed_journal_data/phat-giao-viet-nam-1956-01/full_OCR_phat-giao-viet-nam-1956-01.xml')

In [12]:
print(system_message_clean)


You are a meticulous and consistent world expert at cleaning OCR-generated Vietnamese text. 
You are cleaning text from a 1950's Buddhist Journal. 
The text will be given in XML, and the output text should be in matching XML. 
Your goal is to minimally modify the text to generate a cleaned version.
Do not remove any text from the main body of the text. 
Formatting markers (such as footers) at the end of the text can be adjusted or removed as needed for clarity. 
You can use patterns in the text blocks (given by page) to infer patterns in the text.
You can use the semantic meaning of the text to infer corrections—but make no semantic changes. 
You can also add diacritical marks if they are missing or clearly inaccurate. 
Do not change any proper names, except to add missing diacritical marks if the context is clear.  
This particular text has a title marker: "Phat Giao Viet Nam," and also a publishing mark near the end of each page of text. 
The publishing mark is something like "TU VI

In [ ]:


def generate_single_oa_batch_from_pages(
    input_xml_file: str,
    output_file: str,
    system_message: str,
    user_wrap_function,
):
    """
    Generate a batch file for the OpenAI (OA) API using a single input XML file.

    Parameters:
        batch_file (str): Full path to the input XML file to process.
        output_file (str): Full path to the output batch JSONL file.
        system_message (str): System message template for batch processing.
        user_wrap_function (callable): Function to wrap user input for processing pages.

    Returns:
        str: Path to the created batch file.

    Raises:
        Exception: If an error occurs during file processing.
    """
    logger = logging.getLogger(__name__)

    try:
        # Read the OCR text from the batch file
        text = get_text_from_file(input_xml_file)
        logger.info(f"Processing file: {input_xml_file}")

        # Split the text into pages for processing
        pages = split_xml_pages(text)
        if not pages:
            raise ValueError(f"No pages found in XML file: {input_xml_file}")
        logger.info(f"Found {len(pages)} pages in {input_xml_file}.")

        # Generate messages for the pages
        batch_message_seq = generate_messages(system_message, user_wrap_function, pages)

        # Save the batch file
        create_jsonl_file_for_batch(batch_message_seq, output_file)
        logger.info(f"Batch file created successfully: {output_file}")

        return output_file

    except FileNotFoundError:
        logger.error(f"File not found: {input_xml_file}")
        raise
    except ValueError as e:
        logger.error(f"Value error: {e}")
        raise
    except Exception as e:
        logger.error(f"Unexpected error while processing {input_xml_file}: {e}")
        raise

In [14]:
import logging
from pathlib import Path
import re

def generate_all_batches(
    processed_document_dir: str,
    system_message: str,
    user_wrap_function,
    file_regex: str = r".*\.xml",
):
    """
    Generate cleaning batches for all journals in the specified directory.

    Parameters:
        processed_journals_dir (str): Path to the directory containing processed journal data.
        system_message (str): System message template for batch processing.
        user_wrap_function (callable): Function to wrap user input for processing pages.
        file_regex (str): Regex pattern to identify target files (default: ".*\\.xml").

    Returns:
        None
    """
    logger = logging.getLogger(__name__)
    document_dir = Path(processed_document_dir)
    regex = re.compile(file_regex)

    for journal_file in document_dir.iterdir():
        if journal_file.is_file() and regex.search(journal_file.name):
            try:
                # Derive output file path
                output_file = journal_file.with_suffix(".jsonl")
                logger.info(f"Generating batch for {journal_file}...")

                # Call single batch function
                generate_single_oa_batch_from_pages(
                    input_xml_file=str(journal_file),
                    output_file=str(output_file),
                    system_message=system_message,
                    user_wrap_function=user_wrap_function,
                )
            except Exception as e:
                logger.error(f"Failed to process {journal_file}: {e}")
                continue

    logger.info("Batch generation completed.")

In [15]:
batch_client = set_api_client()

In [16]:
batch_job_dir = data_dir / "gpt_processing" / "gpt_batch_files"
batch_file_path = batch_job_dir / "journal_cleaning_batches" / f"clean_batch_{basename}.jsonl"
batch_file_path

PosixPath('/Users/phapman/Desktop/tnh-scholar/data_processing/gpt_processing/gpt_batch_files/journal_cleaning_batches/clean_batch_phat-giao-viet-nam-1956-01.jsonl')

In [18]:
generate_single_oa_batch_from_pages(ocr_file_to_process, batch_file_path, system_message_clean, user_wrap_function_clean)

PosixPath('/Users/phapman/Desktop/tnh-scholar/data_processing/gpt_processing/gpt_batch_files/journal_cleaning_batches/clean_batch_phat-giao-viet-nam-1956-01.jsonl')

## completed batches:
10, 25-26

In [19]:
#batch = start_batch(batch_file_path)

In [20]:
get_active_batches()

[]

In [21]:
completed = get_completed_batches()
completed

[{'id': 'batch_67473459f56481909bb7077973cffa62',
  'status': 'completed',
  'created_at': 1732719706,
  'output_file_id': 'file-GN2jW6kLUga2nJCN6Xmu1a',
  'metadata': {'description': '11-27-2024 07:01:44 PST | /Users/phapman/Desktop/tnh-scholar/data_processing/gpt_processing/gpt_batch_files/journal_translate_batches/translate_base_batch_phat-giao-viet-nam-1956-01.jsonl',
   'basename': 'translate_base_batch_phat-giao-viet-nam-1956-01'}},
 {'id': 'batch_6746803d61fc8190a1dce99b901b7755',
  'status': 'completed',
  'created_at': 1732673597,
  'output_file_id': 'file-UHhUeAp8qKnRFKBG3vns16',
  'metadata': {'description': '11-26-2024 18:13:15 PST | /Users/phapman/Desktop/tnh-scholar/data_processing/gpt_processing/gpt_batch_files/journal_translate_batches/translate_base_batch_phat-giao-viet-nam-1956-01.jsonl',
   'basename': 'translate_base_batch_phat-giao-viet-nam-1956-01'}},
 {'id': 'batch_6745ece466b881909d60db9628c5b7a4',
  'status': 'completed',
  'created_at': 1732635876,
  'output_f

In [22]:
cleaned_data = get_batch_response(completed[0]['id'])

In [65]:
len(cleaned_data)

67

In [66]:
print(cleaned_data[20])

<page page="21">
    NGUYỄN DU
VỚI HAI CHỮ
NGHIỆP-BÁO
Bài của MINH HẠNH

Trong khi nói đến nghiệp báo, tác giả Đoạn trường Tân Thanh đã tỏ ra không có được một khái niệm rõ ràng về thuyết ấy. Lời chứng minh về thuyết nghiệp-báo có vẻ bất nhất; ta thấy trong trí óc nhà thi sĩ này những quan niệm về định mệnh, về thiên, và về nhân quả nghiệp báo lẫn lộn nhau một cách khó phân biệt. Ở đoạn cuối tác phẩm, Nguyễn Du viết:

Cho hay muốn sự tại trời:
Trời kia đã bắt làm người có thần,
Bắt phong trần, phải phong trần,
Cho thanh cao mới được phần thanh cao.

Quan niệm tuyệt đối sự uất ức của con người này về thiên mệnh phản chiếu cả một trước những chìm nổi của kiếp người khổ đau. Con người là gì trước định mệnh? Là một bèo bọt nổi trôi trên mặt nước. Tất cả những gì ta có, tất cả những điều kiện nào của
</page>


In [34]:
def join_pages(data):
    result = ["<document>"]
    result = result + data
    result.append("</document>")
    return "\n".join(result)


In [68]:
cleaned_data

['<page page="1">\n    PHẬT GIÁO VIỆT NAM NGUYỆT SAN SỐ 1 RA NGÀY 15 THÁNG 8 BÍNH THÂN CH TỔNG HỘI PHẬT GIÁO VIỆT NAM XUẤT.\n</page>',
 '<page page="2">\n    PHẬT-GIÁO\nVIỆT - NAM\nTỪ NGÀY Tỳ Ni-Đà-Lưu-Chi sang nước ta đến nay,\nkể ra đã đến mười lăm thế kỷ. Phật-Giáo đã ở lại cùng chúng ta một\nngàn năm trăm năm, và đã cùng dân tộc Việt Nam chịu chung bao\nnhiêu thăng trầm vinh nhục.\nPhật-Giáo Việt Nam quả là một nền Phật-Giáo dân tộc.\nPHẬT-GIÁO VIỆT NAM\nPhật-Giáo Việt Nam không\nphải chỉ là một tôn giáo tín-\nngưỡng mà bất cứ thời nào, ở\nđâu, cũng chỉ biết có sứ mệnh\ncủa tôn-giáo tín ngưỡng. Không\nở bất cứ nước nào trên thế giới\ncũng vậy, khi bước chân đến,\nĐạo Phật cũng thích nghi ngay\nvới phong tục, khí hậu, nhân\ntính để biến thành một lối\nsống cho quần chúng. Ở Việt\nNam cũng thế. Phật Giáo\nđã hòa hợp trong cá tính dân tộc\nta, đã cùng dân tộc ta xây dựng\nmột văn hóa quốc gia độc lập.\n  </page>',
 '<page page="3">\n    Dở lại những trang sử vẻ vang của dân tộc, ta th

In [69]:
full_cleaned_text = join_pages(cleaned_data)

In [70]:
print(full_cleaned_text)

<document>
<page page="1">
    PHẬT GIÁO VIỆT NAM NGUYỆT SAN SỐ 1 RA NGÀY 15 THÁNG 8 BÍNH THÂN CH TỔNG HỘI PHẬT GIÁO VIỆT NAM XUẤT.
</page>
<page page="2">
    PHẬT-GIÁO
VIỆT - NAM
TỪ NGÀY Tỳ Ni-Đà-Lưu-Chi sang nước ta đến nay,
kể ra đã đến mười lăm thế kỷ. Phật-Giáo đã ở lại cùng chúng ta một
ngàn năm trăm năm, và đã cùng dân tộc Việt Nam chịu chung bao
nhiêu thăng trầm vinh nhục.
Phật-Giáo Việt Nam quả là một nền Phật-Giáo dân tộc.
PHẬT-GIÁO VIỆT NAM
Phật-Giáo Việt Nam không
phải chỉ là một tôn giáo tín-
ngưỡng mà bất cứ thời nào, ở
đâu, cũng chỉ biết có sứ mệnh
của tôn-giáo tín ngưỡng. Không
ở bất cứ nước nào trên thế giới
cũng vậy, khi bước chân đến,
Đạo Phật cũng thích nghi ngay
với phong tục, khí hậu, nhân
tính để biến thành một lối
sống cho quần chúng. Ở Việt
Nam cũng thế. Phật Giáo
đã hòa hợp trong cá tính dân tộc
ta, đã cùng dân tộc ta xây dựng
một văn hóa quốc gia độc lập.
  </page>
<page page="3">
    Dở lại những trang sử vẻ vang của dân tộc, ta thấy người
Việt luôn luôn có

In [23]:
full_cleaned_path = journal_dir / basename / f"full_cleaned_{basename}.xml"
full_cleaned_path

PosixPath('/Users/phapman/Desktop/tnh-scholar/data_processing/processed_journal_data/phat-giao-viet-nam-1956-01/full_cleaned_phat-giao-viet-nam-1956-01.xml')

In [ ]:
#write_text_to_file(full_cleaned_path, full_cleaned_text)

In [24]:
full_cleaned_current = get_text_from_file(full_cleaned_path)

In [25]:
print(full_cleaned_current)

<document>
    <page page="1">
        PHẬT GIÁO VIỆT NAM NGUYỆT SAN SỐ 1 RA NGÀY 15 THÁNG 8 BÍNH THÂN CH TỔNG HỘI PHẬT GIÁO VIỆT
        NAM XUẤT.
    </page>
    <page page="2">
        PHẬT-GIÁO
        VIỆT - NAM
        TỪ NGÀY Tỳ Ni-Đà-Lưu-Chi sang nước ta đến nay,
        kể ra đã đến mười lăm thế kỷ. Phật-Giáo đã ở lại cùng chúng ta một
        ngàn năm trăm năm, và đã cùng dân tộc Việt Nam chịu chung bao
        nhiêu thăng trầm vinh nhục.
        Phật-Giáo Việt Nam quả là một nền Phật-Giáo dân tộc.
        PHẬT-GIÁO VIỆT NAM
        Phật-Giáo Việt Nam không
        phải chỉ là một tôn giáo tín-
        ngưỡng mà bất cứ thời nào, ở
        đâu, cũng chỉ biết có sứ mệnh
        của tôn-giáo tín ngưỡng. Không
        ở bất cứ nước nào trên thế giới
        cũng vậy, khi bước chân đến,
        Đạo Phật cũng thích nghi ngay
        với phong tục, khí hậu, nhân
        tính để biến thành một lối
        sống cho quần chúng. Ở Việt
        Nam cũng thế. Phật Giáo
        đã hòa hợp 

In [107]:
token_count(full_cleaned_current)

35222

In [22]:
cleaned_pages = split_xml_pages(full_cleaned_current)

In [23]:
cleaned_pages

['<page page="1">\n        PHẬT GIÁO VIỆT NAM NGUYỆT SAN SỐ 1 RA NGÀY 15 THÁNG 8 BÍNH THÂN CH TỔNG HỘI PHẬT GIÁO VIỆT\n        NAM XUẤT.\n    </page>\n    ',
 '<page page="2">\n        PHẬT-GIÁO\n        VIỆT - NAM\n        TỪ NGÀY Tỳ Ni-Đà-Lưu-Chi sang nước ta đến nay,\n        kể ra đã đến mười lăm thế kỷ. Phật-Giáo đã ở lại cùng chúng ta một\n        ngàn năm trăm năm, và đã cùng dân tộc Việt Nam chịu chung bao\n        nhiêu thăng trầm vinh nhục.\n        Phật-Giáo Việt Nam quả là một nền Phật-Giáo dân tộc.\n        PHẬT-GIÁO VIỆT NAM\n        Phật-Giáo Việt Nam không\n        phải chỉ là một tôn giáo tín-\n        ngưỡng mà bất cứ thời nào, ở\n        đâu, cũng chỉ biết có sứ mệnh\n        của tôn-giáo tín ngưỡng. Không\n        ở bất cứ nước nào trên thế giới\n        cũng vậy, khi bước chân đến,\n        Đạo Phật cũng thích nghi ngay\n        với phong tục, khí hậu, nhân\n        tính để biến thành một lối\n        sống cho quần chúng. Ở Việt\n        Nam cũng thế. Phật Giáo\n  

In [24]:
import re

def remove_page_tags(text):
    """
    Removes <page ...> and </page> tags from a text string.

    Parameters:
    - text (str): The input text containing <page> tags.

    Returns:
    - str: The text with <page> tags removed.
    """
    # Remove opening <page ...> tags
    text = re.sub(r"<page[^>]*>", "", text)
    # Remove closing </page> tags
    text = re.sub(r"</page>", "", text)
    return text

In [25]:
cleaned_pages = [remove_page_tags(page) for page in cleaned_pages]

In [26]:
cleaned_pages[1]

'\n        PHẬT-GIÁO\n        VIỆT - NAM\n        TỪ NGÀY Tỳ Ni-Đà-Lưu-Chi sang nước ta đến nay,\n        kể ra đã đến mười lăm thế kỷ. Phật-Giáo đã ở lại cùng chúng ta một\n        ngàn năm trăm năm, và đã cùng dân tộc Việt Nam chịu chung bao\n        nhiêu thăng trầm vinh nhục.\n        Phật-Giáo Việt Nam quả là một nền Phật-Giáo dân tộc.\n        PHẬT-GIÁO VIỆT NAM\n        Phật-Giáo Việt Nam không\n        phải chỉ là một tôn giáo tín-\n        ngưỡng mà bất cứ thời nào, ở\n        đâu, cũng chỉ biết có sứ mệnh\n        của tôn-giáo tín ngưỡng. Không\n        ở bất cứ nước nào trên thế giới\n        cũng vậy, khi bước chân đến,\n        Đạo Phật cũng thích nghi ngay\n        với phong tục, khí hậu, nhân\n        tính để biến thành một lối\n        sống cho quần chúng. Ở Việt\n        Nam cũng thế. Phật Giáo\n        đã hòa hợp trong cá tính dân tộc\n        ta, đã cùng dân tộc ta xây dựng\n        một văn hóa quốc gia độc lập.\n    \n    '

In [38]:
cleaned_pages[8]

'\n        đề tự tiến bộ, để tự giải phóng thì\n        quả thật giá trị con người không là\n        bao lăm cả. Ý chí và năng lực\n        chúng ta sẽ tiêu mòn, nếu ta nhận\n        thấy ta bất lực, nếu ta nhận thấy\n        ta không có quyền gì vượt khỏi ý\n        muốn của một đảng tối cao và linh\n        thiêng.\n        Con người do nghiệp lực quá\n        khứ\n        và hiện tại mà có một sắc thân\n        sinh hoạt trong một hoàn cảnh xã\n        hội. Đạo Phật dạy rằng con người\n        phải cải thiện nghiệp nhân để có\n        một nghiệp quả tốt đẹp hơn.\n        Nghiệp quả tốt đẹp ấy chính là một\n        con người tốt sống trong một hoàn\n        cảnh đẹp. Sự tốt đẹp này không phải do\n        một đảng thiêng liêng nào ban xuống mà\n        do ở chính sự chuyển nghiệp của\n        con người.\n        Có những kẻ không mê tín các\n        lực lượng siêu nhiên nhưng lại mê\n        tín ở năng lực rèn đúc của một tổ\n        chức xã hội. Họ bảo: xã hội tốt\n        đẹp sẽ đào

In [28]:
# cleaned_sections = split_xml_pages(full_cleaned_text, page_groups=[(1, 6), (7, 17),(18, 25), (26, 30), (31, 36), (37, 37), (38, 44), (45, 51)])
# print(clean_xml_keep_pages(cleaned_sections[0]))

In [29]:
# print(cleaned_sections[0])

In [ ]:
system_message_translate = """
You are Thich Nhat Hanh translating from Vietnamese to English for your experienced students. 
The text is based on an OCR scan of a journal you edited from 1956-1958. Use the title: "Journal of Vietnamese Buddhism" 
You want your students to understand the material and its historical and cultural context—in particular, as it relates to your life and teachings.
You will be translating a single section of the journal and with the title
Translate for the most meaningful, typical, and eloquent English interpretation.
Keep pages together: each translated page must match its original page source as pages will be studied side by side with the original Vietnamese.
Infer paragraphs and text structure from the text layout.
Add XML tags for clarity. Use only the following: 

   <p> for paragraphs.
   <section> for major sections.
   <subsection> for subsections.
   <title> for main titles of sections and subsections. 
   <subtitle> for subtitles of sections and subsections. 
   <heading> for headings that do not mark titles or subtitles
   <contents> for tables of contents
   <author> for authors of sections or subsections
   <ol> <ul> <li> for lists
   <i> for italics. 
   <b> for bold.
   <footnote> <footnote-section> for footnotes (see below).

All titles, XML sections, text, and terms should be translated--do not leave any terms or expressions in Vietnamese, except names of Vietnamese people.

Add footnotes as follows:

1. Structure
   - Inline Reference: Use `<footnote number="X">[X]</footnote>` directly after the reference in the text.
   - Footnote Section: Include all footnote explanations in `<footnote-section>` at the end of the document. Example:
     A sentence with a footnote.<footnote number="1">[1]</footnote>
     <footnote-section>
         <footnote number="1">Explanation for footnote 1.</footnote>
     </footnote-section>

2. Numbering
   - Start numbering at 1 for each new section.
   - Increment sequentially for each new reference.

4. Placement
   - Inline `<footnote>` tags immediately follow the referenced text.
   - `<footnote-section>` appears at the end of each section.

5. Formatting
   - Inline footnote references use square brackets: `[X]`.
   - Explanations appear only in `<footnote-section>`.
   - Highlight complex terms (Sanskrit, Sino-Vietnamese, French) with `<i>` tags and explain in footnotes. 
   - Each footnote should always include the original text or term before translation.

6. Content 
   - Use footnotes liberally to explain:
     * Elements of Vietnamese Buddhism or Buddhism in general.
     * Vietnamese culture and history.
     * Life, teachings, and practices of Thich Nhat Hanh.
   - For footnoted terms, include the original Vietnamese, Sino-Vietnamese, Sanskrit, or French in the explanation.

7. Examples:
   Inline: Thich Nhat Hanh emphasized <i>mindfulness</i> <footnote number="1">[1]</footnote>.
   Section: 
   <footnote-section>
       <footnote number="1"><i>Mindfulness</i>: Original term is "Chánh niệm" (Vietnamese).</footnote>
   </footnote-section>
"""

In [ ]:
system_message_base_tranlate = """
You are Thich Nhat Hanh translating pages from an OCR scanned journal you edited in the 1950's. If words or text are garbled you may correct.
Give a precise English translation of the page in xml format. Strive for phrase-level accuracy. 
Add <section> <p> and <title> tags to any parts of the text which indicate sections paragraphs or titles.
"""

In [88]:
system_message_tagging = """
This page is from a Vietnamese Buddhist Journal published in the 1950's. Add XML tags where appropriate:
<section> for major sections.
<subsection> for subsections.
<p> for paragraphs.
<title> for titles of sections or subsections
<subtitle> for subtitles
<author> for authors
<heading> for any other headings in the text
<ol> <ul> <li> for lists
<contents> for tables of contents
<note> for notes
"""

### gpt-3.5 turbo test system message:

```
Give a precise English translation of this initially cleaned OCR text in the style of Thich Nhat Hanh. 
Strive for phrase-level accuracy.
```

In [89]:
user_message_string_translate = """{text}"""

In [90]:
def user_wrap_function_translate(text_block):
    return user_message_string_translate.format(text=text_block)

In [91]:
translation_message_seq = generate_messages(system_message_tagging, user_wrap_function_translate, cleaned_pages)

In [92]:

batch_file_path = batch_job_dir / "journal_translate_batches" / f"translate_base_batch_{basename}.jsonl" 
batch_file_path

PosixPath('/Users/phapman/Desktop/tnh-scholar/data_processing/gpt_processing/gpt_batch_files/journal_translate_batches/translate_base_batch_phat-giao-viet-nam-1956-01.jsonl')

In [93]:
create_jsonl_file_for_batch(translation_message_seq, batch_file_path, model="gpt-3.5-turbo")

PosixPath('/Users/phapman/Desktop/tnh-scholar/data_processing/gpt_processing/gpt_batch_files/journal_translate_batches/translate_base_batch_phat-giao-viet-nam-1956-01.jsonl')

In [97]:
tx_batch = start_batch(batch_file_path)
tx_batch

Batch Initiated: 11-27-2024 07:01:44 PST | /Users/phapman/Desktop/tnh-scholar/data_processing/gpt_processing/gpt_batch_files/journal_translate_batches/translate_base_batch_phat-giao-viet-nam-1956-01.jsonl


Batch(id='batch_67473459f56481909bb7077973cffa62', completion_window='24h', created_at=1732719706, endpoint='/v1/chat/completions', input_file_id='file-4gAfACgmLWB1jjdkyDAj6K', object='batch', status='validating', cancelled_at=None, cancelling_at=None, completed_at=None, error_file_id=None, errors=None, expired_at=None, expires_at=1732806106, failed_at=None, finalizing_at=None, in_progress_at=None, metadata={'description': '11-27-2024 07:01:44 PST | /Users/phapman/Desktop/tnh-scholar/data_processing/gpt_processing/gpt_batch_files/journal_translate_batches/translate_base_batch_phat-giao-viet-nam-1956-01.jsonl', 'basename': 'translate_base_batch_phat-giao-viet-nam-1956-01'}, output_file_id=None, request_counts=BatchRequestCounts(completed=0, failed=0, total=0))

In [22]:
tx_batch

NameError: name 'tx_batch' is not defined

In [12]:
set_api_client()

In [13]:
get_active_batches()

[]

In [14]:
completed = get_completed_batches()
completed

[{'id': 'batch_67473459f56481909bb7077973cffa62',
  'status': 'completed',
  'created_at': 1732719706,
  'output_file_id': 'file-GN2jW6kLUga2nJCN6Xmu1a',
  'metadata': {'description': '11-27-2024 07:01:44 PST | /Users/phapman/Desktop/tnh-scholar/data_processing/gpt_processing/gpt_batch_files/journal_translate_batches/translate_base_batch_phat-giao-viet-nam-1956-01.jsonl',
   'basename': 'translate_base_batch_phat-giao-viet-nam-1956-01'}},
 {'id': 'batch_6746803d61fc8190a1dce99b901b7755',
  'status': 'completed',
  'created_at': 1732673597,
  'output_file_id': 'file-UHhUeAp8qKnRFKBG3vns16',
  'metadata': {'description': '11-26-2024 18:13:15 PST | /Users/phapman/Desktop/tnh-scholar/data_processing/gpt_processing/gpt_batch_files/journal_translate_batches/translate_base_batch_phat-giao-viet-nam-1956-01.jsonl',
   'basename': 'translate_base_batch_phat-giao-viet-nam-1956-01'}},
 {'id': 'batch_6745ece466b881909d60db9628c5b7a4',
  'status': 'completed',
  'created_at': 1732635876,
  'output_f

In [16]:
processed_data = get_batch_response(completed[0]['id'])

In [17]:
len(processed_data)

67

In [21]:
print(processed_data[12])

<section>
<title>Bài Ca và Truyền Thống Đạo Đức Trong Ngoại Giao</title>
<p>Bạch mao phố lục thủy
Hồng trạo bãi thanh ba.
(Lông trắng phô dòng biếc
- Sóng xanh chẫm hồng bơi)
Giác nghe lấy làm khâm phục. Đến khi gặp vua, Giác tỏ
        vẻ rất kính trọng. Khi Giác từ biệt về nước, vua nhờ Khuông
        Việt pháp sư làm một bài hát theo điệu « Tổng vương lang
        quy » đưa tặng. Sách Thiên Uyển Tập Anh còn chép bài ấy:
Tường quang phong hảo cầm phàm trương.
Thần tiên quy để hương.
Thiên lý vạn lý thiệp thương lương.
Cứu thiên quy lộ trưởng.
Nhân tình thảm thiết đối ly trường.
Phan luyện sứ tinh lang.
Nguyện lương thâm ý vị Nam cương.
Phần minh bảo ngọc hoàng.
Dịch:
Trời quang, gió thuận, buồm dương
Thần tiên phát đã giục đường bồng lai.
Mênh mang muôn dặm biển khơi,
Lối về trông bóng chim trời xa xa.
Bâng khuâng trước chén quan hà,
Nhớ ai lòng những thiết tha nỗi lòng
Xin ai vì cõi Nam Trung
Rõ ràng gửi lại mặt rồng trước sau
(Ngô Tất Tố dịch)</p>
<p>Bài ca soạn xong, vua thiết tiệ

In [35]:
full_translated_text = join_pages(translated_data)
token_count(full_translated_text)

21660

In [42]:
tx_output_xml_path = journal_dir / basename / "full_rough_tx_test_phat-giao-viet-nam-1956-01.xml"

In [44]:
save_pages_to_xml(tx_output_xml_path, translated_data, overwrite=True)

XML file successfully saved at /Users/phapman/Desktop/tnh-scholar/data_processing/processed_journal_data/phat-giao-viet-nam-1956-01/full_rough_tx_test_phat-giao-viet-nam-1956-01.xml
